In [ ]:
using StateSpaceDynamics
using Random
using Distributions
using Plots
using LinearAlgebra
using ForwardDiff
using StatsFuns
using Optim



ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


In [ ]:
# generate synthetic data for a switching Gaussian Regression
Random.seed!(12345)

n = 1000

beta_1 = [1.0, 2.0]
beta_2 = [3.0, 4.0]

sigma_1 = 0.5
sigma_2 = 0.5

# Simualte data from a switching GLM
A = [0.9 0.1; 0.1 0.9]
pi_k = [0.5, 0.5]

X = randn(n, 2)
Y = zeros(n)
states = zeros(Int, n)

# First state
states[1] = Int(rand() < pi_k[1] ? 1 : 2)
Y[1] = if states[1] == 1
    beta_1[1] + beta_1[2] * X[1, 1] + sigma_1 * randn()
else
    beta_2[1] + beta_2[2] * X[1, 1] + sigma_2 * randn()
end

# Remaining states
for i in 2:n
    states[i] = Int(rand() < A[states[i-1], 1] ? 1 : 2)
    Y[i] = if states[i] == 1
        beta_1[1] + beta_1[2] * X[i, 1] + sigma_1 * randn()
    else
        beta_2[1] + beta_2[2] * X[i, 1] + sigma_2 * randn()
    end
end


In [ ]:
# reshape x and y to be matrices
X_input = reshape(X[:, 1], 1, n)
Y = reshape(Y, 1, n)

est_model = SwitchingGaussianRegression(K=2, input_dim=1, output_dim=1)
fit!(est_model, X_input, Y)

New Function in use...


CompositeException: TaskFailedException

    nested task error: DimensionMismatch: A has dimensions (1,1001) but B has dimensions (2,1)
    Stacktrace:
      [1] gemm_wrapper!(C::Matrix{Float64}, tA::Char, tB::Char, A::Matrix{Float64}, B::Matrix{Float64}, _add::LinearAlgebra.MulAddMul{true, true, Bool, Bool})
        @ LinearAlgebra C:\Users\ryansenne\.julia\juliaup\julia-1.10.4+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:577
      [2] generic_matmatmul!
        @ C:\Users\ryansenne\.julia\juliaup\julia-1.10.4+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:352 [inlined]
      [3] mul!
        @ C:\Users\ryansenne\.julia\juliaup\julia-1.10.4+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:263 [inlined]
      [4] mul!
        @ C:\Users\ryansenne\.julia\juliaup\julia-1.10.4+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:237 [inlined]
      [5] *(A::Matrix{Float64}, B::Matrix{Float64})
      

In [ ]:
# generate synthetic data
Random.seed!(123)

# set parameters
n = 1000
β = [0.1, 0.9]

# generate data
X = hcat(ones(n), randn(n, 1))

p = logistic.(X * β)
y = rand.(Bernoulli.(p));

logreg = SSM.BernoulliRegression()
SSM.fit!(logreg, reshape(X[:, 2], length(y), 1), y)

In [ ]:
model = SSM.SwitchingBernoulliRegression(;K=2, λ=0.1)
SSM.fit!(model, reshape(X[:, 2], length(y), 1), y, 10)

In [ ]:
# synthetic poisson regression data
Random.seed!(123)

# set parameters
n = 100
β = [0.1, 0.9]
# generate data

X = hcat(ones(n), randn(n, 1))
λ = exp.(X * β)

y = rand.(Poisson.(λ))

poisreg = SSM.PoissonRegression()
SSM.fit!(poisreg, reshape(X[:, 2], length(y), 1), y)

In [ ]:
model = SSM.SwitchingPoissonRegression(;K=2, λ=0.1)
fit!(model, reshape(X[:, 2], length(y), 1), y)

In [ ]:
# Parameters
n = 1000
β₁ = [0.1, 0.9]
β₂ = [0.9, 0.1]
σ²₁ = 0.1
σ²₂ = 0.1
A = [0.9 0.1; 0.1 0.9]

# Data generation
X = hcat(ones(n), randn(n, 1))
y = zeros(n)
state = Vector{Int}(undef, n)

for i in 1:n
    if i == 1
        # choose a state at random
        state[i] = rand(1:2)
    else
        # Transition based on previous state
        state[i] = rand(Categorical(A[state[i-1], :]))
    end

    if state[i] == 1
        y[i] = X[i, :]' * β₁ + sqrt(σ²₁) * randn()
    else
        y[i] = X[i, :]' * β₂ + sqrt(σ²₂) * randn()
    end
end

# Fit linear regression model
linreg = SSM.SwitchingGaussianRegression(; K=2, λ=0.1)
SSM.fit!(linreg, reshape(X[:, 2], length(y), 1), y, 10000, 1e-6)

println("Fitted coefficients: ", linreg.B[1].regression.β)


In [ ]:
γ, ξ, α, β = SSM.E_step(linreg, reshape(X[:, 2], length(y), 1), y)